In [1]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [2]:
# Function to generate the search URL
def generate_url(job_title, location, page_number=1):
    job_title_encoded = job_title.replace(" ", "-")
    location_encoded = location.replace(" ", "-")
    return f'https://www.naukri.com/{job_title_encoded}-jobs-in-{location_encoded}-{page_number}?k={job_title_encoded}&l={location_encoded}'
#In many URLs,spaces are typically encoded as either hyphens ("-") or plus signs ("+").The behavior of using either can depend on how the target website processes the URL parameters.

In [3]:
# Function to scrape job listings from the current page
def scrape_jobs(driver, job_data):
    job_listings = driver.find_elements(By.CLASS_NAME, 'srp-jobtuple-wrapper')
    for listing in job_listings:
        try:
            job_title_element = listing.find_element(By.CLASS_NAME, 'title')
            job_title = job_title_element.text.strip()

            company_name_element = listing.find_element(By.CLASS_NAME, 'comp-name')
            company_name = company_name_element.text.strip()

            experience_element = listing.find_element(By.CLASS_NAME, 'exp-wrap')
            experience = experience_element.text.strip()

            salary_element = listing.find_element(By.CLASS_NAME, 'sal-wrap')
            salary = salary_element.text.strip()

            location_element = listing.find_element(By.CLASS_NAME, 'loc-wrap')
            location = location_element.text.strip()

            posted_day_element = listing.find_element(By.CLASS_NAME, 'job-post-day')
            posted = posted_day_element.text.strip()

            # Append the job data to the list
            job_data.append({
                'Job Title': job_title,
                'Company Name': company_name,
                'Experience Required': experience,
                'Salary': salary,
                'Location': location,
                'Posted': posted
            })
        except Exception as e:
            print(f"Error extracting job listing: {e}")
    
    return job_data

In [4]:
# Function to scrape job data for given job titles and locations
def scrape_job_data(job_titles, locations, max_pages=200):
    # Initialize the WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    all_job_data = []

    for job_title in job_titles:
        for location in locations:
            print(f"Scraping jobs for {job_title} in {location}...")
            page = 1

            while page <= max_pages:
                url = generate_url(job_title, location, page)
                print(f"Scraping URL: {url}")
                driver.get(url)
                time.sleep(5)  # Adjust the sleep time as necessary

                all_job_data = scrape_jobs(driver, all_job_data)
                
                # If no jobs are found on the page, break out of the loop
                if not driver.find_elements(By.CLASS_NAME, 'srp-jobtuple-wrapper'):
                    print(f"No more job listings found on page {page}. Stopping.")
                    break

                page += 1

    # Close the WebDriver
    driver.quit()

    return all_job_data

In [7]:
job_titles = ['data-analyst', 'data-scientist', 'data-engineer']
locations = ['india']#['mumbai', 'pune', 'bangalore']
#max_pages = 150

job_data = scrape_job_data(job_titles, locations)#, max_pages)

# Convert the list to a pandas DataFrame
df = pd.DataFrame(job_data)

# Optionally save the DataFrame to a CSV file
df.to_csv('naukri_job_listings.csv', index=False)

# Display the DataFrame
print(df.head())

Scraping jobs for data-analyst in india...
Scraping URL: https://www.naukri.com/data-analyst-jobs-in-india-1?k=data-analyst&l=india
Scraping URL: https://www.naukri.com/data-analyst-jobs-in-india-2?k=data-analyst&l=india
Scraping URL: https://www.naukri.com/data-analyst-jobs-in-india-3?k=data-analyst&l=india
Scraping URL: https://www.naukri.com/data-analyst-jobs-in-india-4?k=data-analyst&l=india
Scraping URL: https://www.naukri.com/data-analyst-jobs-in-india-5?k=data-analyst&l=india
Scraping URL: https://www.naukri.com/data-analyst-jobs-in-india-6?k=data-analyst&l=india
Scraping URL: https://www.naukri.com/data-analyst-jobs-in-india-7?k=data-analyst&l=india
Scraping URL: https://www.naukri.com/data-analyst-jobs-in-india-8?k=data-analyst&l=india
Scraping URL: https://www.naukri.com/data-analyst-jobs-in-india-9?k=data-analyst&l=india
Scraping URL: https://www.naukri.com/data-analyst-jobs-in-india-10?k=data-analyst&l=india
Scraping URL: https://www.naukri.com/data-analyst-jobs-in-india-11